In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 58.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = "/content/drive/MyDrive/HIGGS.csv"

df = pd.read_csv(file_path, nrows=10000, header=None)
print("Shape of loaded dataset:", df.shape)

Shape of loaded dataset: (10000, 29)


In [ ]:
y = df.iloc[:, 0].values
X = df.iloc[:, 1:].values
print("Feature matrix X shape:", X.shape)
print("Label vector y shape:", y.shape)

Feature matrix X shape: (10000, 28)
Label vector y shape: (10000,)


In [ ]:
print(df.iloc[:5, :28])

    0         1         2         3         4         5         6         7   \
0  1.0  0.869293 -0.635082  0.225690  0.327470 -0.689993  0.754202 -0.248573   
1  1.0  0.907542  0.329147  0.359412  1.497970 -0.313010  1.095531 -0.557525   
2  1.0  0.798835  1.470639 -1.635975  0.453773  0.425629  1.104875  1.282322   
3  0.0  1.344385 -0.876626  0.935913  1.992050  0.882454  1.786066 -1.646778   
4  1.0  1.105009  0.321356  1.522401  0.882808 -1.205349  0.681466 -1.070464   

         8         9   ...        18        19        20        21        22  \
0 -1.092064  0.000000  ...  0.657930 -0.010455 -0.045767  3.101961  1.353760   
1 -1.588230  2.173076  ...  0.398701 -1.138930 -0.000819  0.000000  0.302220   
2  1.381664  0.000000  ...  1.256955  1.128848  0.900461  0.000000  0.909753   
3 -0.942383  0.000000  ...  0.745313 -0.678379 -1.360356  0.000000  0.946652   
4 -0.921871  0.000000  ...  0.479999 -0.373566  0.113041  0.000000  0.755856   

         23        24        25       

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 75.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math
from sklearn.model_selection import train_test_split

class SineKANLayer(nn.Module):
    def __init__(self, input_dim, output_dim, grid_size=5):
        super().__init__()
        self.grid_size = grid_size
        self.A, self.K, self.C = 0.9724, 0.9884, 0.9994
        self.grid_norm = (torch.arange(grid_size, dtype=torch.float32) + 1).reshape(1, 1, grid_size)
        amp = torch.empty(output_dim, input_dim, 1).normal_(0, .4) / output_dim / self.grid_norm
        self.amplitudes = nn.Parameter(amp.float())
        self.freq = nn.Parameter(torch.arange(1, grid_size + 1).float().reshape(1, 1, 1, grid_size))
        grid_phase = torch.arange(1, grid_size + 1, dtype=torch.float32).reshape(1, 1, 1, grid_size) / (grid_size + 1)
        input_phase = torch.linspace(0, math.pi, input_dim, dtype=torch.float32).reshape(1, 1, input_dim, 1)
        phase = grid_phase + input_phase
        for _ in range(1, grid_size):
            phase = (self.A * grid_size ** (-self.K) + self.C) * phase
        self.register_buffer("phase", phase)
        self.bias = nn.Parameter(torch.ones(1, output_dim).float() / output_dim)

    def forward(self, x):
        x = x.to(self.amplitudes.dtype)
        x = x.view(x.shape[0], -1).unsqueeze(-1).unsqueeze(1)
        s = torch.sin(x * self.freq + self.phase)
        y = torch.einsum('bijd,ojd->bo', s, self.amplitudes)
        return y + self.bias

class SineKAN(nn.Module):
    def __init__(self, input_dim, hidden=64, output_dim=2):
        super().__init__()
        self.l1 = SineKANLayer(input_dim, hidden)
        self.l2 = SineKANLayer(hidden, output_dim)

    def forward(self, x):
        return self.l2(self.l1(x))

def load_higgs_data(path, Nsamples=1000):
    df = pd.read_csv(path, header=None)
    df = df.sample(frac=1.0, random_state=42)
    X = df.iloc[:Nsamples, 1:].values
    y = df.iloc[:Nsamples, 0].values
    X_min = X.min(axis=1, keepdims=True)
    X_max = X.max(axis=1, keepdims=True)
    X_scaled = 2 * (X - X_min) / (X_max - X_min + 1e-8) - 1
    return train_test_split(
        torch.tensor(X_scaled, dtype=torch.float32),
        torch.tensor(y, dtype=torch.long),
        test_size=0.2,
        stratify=y,
        random_state=42
    )

def train_sinekan_higgs(file_path):
    X_train, X_test, y_train, y_test = load_higgs_data(file_path, Nsamples=1000)
    input_dim = X_train.shape[1]
    model = SineKAN(input_dim=input_dim, hidden=14, output_dim=2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    X_train = X_train.to(device)
    X_test = X_test.to(device)
    y_train = y_train.to(device)
    y_test = y_test.to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        preds = model(X_train)
        loss = criterion(preds, y_train)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            acc = (model(X_test).argmax(dim=1) == y_test).float().mean().item()
        print(f"[SineKAN HIGGS] Epoch {epoch+1:02d} | Loss: {loss.item():.4f} | Test Acc: {acc:.4f}")

train_sinekan_higgs(file_path)


[SineKAN HIGGS] Epoch 01 | Loss: 0.9329 | Test Acc: 0.4800
[SineKAN HIGGS] Epoch 02 | Loss: 1.0716 | Test Acc: 0.4750
[SineKAN HIGGS] Epoch 03 | Loss: 0.6972 | Test Acc: 0.4950
[SineKAN HIGGS] Epoch 04 | Loss: 0.6962 | Test Acc: 0.5150
[SineKAN HIGGS] Epoch 05 | Loss: 0.6999 | Test Acc: 0.5100
[SineKAN HIGGS] Epoch 06 | Loss: 0.7005 | Test Acc: 0.5100
[SineKAN HIGGS] Epoch 07 | Loss: 0.6927 | Test Acc: 0.5350
[SineKAN HIGGS] Epoch 08 | Loss: 0.6847 | Test Acc: 0.5550
[SineKAN HIGGS] Epoch 09 | Loss: 0.6764 | Test Acc: 0.5850
[SineKAN HIGGS] Epoch 10 | Loss: 0.6694 | Test Acc: 0.5950
[SineKAN HIGGS] Epoch 11 | Loss: 0.6646 | Test Acc: 0.5800
[SineKAN HIGGS] Epoch 12 | Loss: 0.6616 | Test Acc: 0.5600
[SineKAN HIGGS] Epoch 13 | Loss: 0.6593 | Test Acc: 0.5450
[SineKAN HIGGS] Epoch 14 | Loss: 0.6572 | Test Acc: 0.5550
[SineKAN HIGGS] Epoch 15 | Loss: 0.6543 | Test Acc: 0.5750
[SineKAN HIGGS] Epoch 16 | Loss: 0.6498 | Test Acc: 0.5550
[SineKAN HIGGS] Epoch 17 | Loss: 0.6438 | Test Acc: 0.55

In [ ]:
import pennylane as qml

class QSVTLayerMulti(nn.Module):
    def __init__(self, coeff_list):
        super().__init__()
        self.coeff_list = nn.ParameterList([
            nn.Parameter(torch.tensor(coeff, dtype=torch.float32), requires_grad=True)
            for coeff in coeff_list
        ])
        self.theta = nn.Parameter(torch.randn(1))
        self.dev = qml.device("default.qubit", wires=1)

        @qml.qnode(self.dev, interface="torch", diff_method="backprop")
        def circuit(x_scalar, theta_val, coeffs_val):
            x_scalar = torch.clamp(x_scalar, -1.0, 1.0)
            qml.RY(x_scalar, wires=0)
            A = x_scalar.detach().cpu().numpy().reshape(1, 1)
            coeffs_np = coeffs_val.detach().cpu().numpy()
            qml.qsvt(A, coeffs_np, encoding_wires=[0], block_encoding="embedding")
            qml.RZ(theta_val[0], wires=0)
            return qml.expval(qml.PauliZ(0))

        self.circuit = circuit

    def forward(self, x):
        B, D = x.shape
        poly_outputs = []
        for coeffs in self.coeff_list:
            outputs = []
            for i in range(D):
                out_i = [self.circuit(x[b, i], self.theta, coeffs) for b in range(B)]
                outputs.append(torch.stack(out_i))
            poly_outputs.append(torch.stack(outputs, dim=1))  # [B, D]
        return torch.stack(poly_outputs, dim=2)  # [B, D, P]

def make_quantum_lcu_layer():
    dev = qml.device("default.qubit", wires=3)

    @qml.qnode(dev, interface="torch", diff_method="backprop")
    def lcu_circuit(val, weight):
        qml.RY(val, wires=0)
        qml.RY(weight, wires=1)
        qml.CNOT(wires=[0, 2])
        angle = torch.clamp(val * weight, -math.pi, math.pi)
        qml.ctrl(qml.RZ, control=0)(angle, wires=2)
        return qml.expval(qml.PauliZ(2))

    def forward_fn(qsvt_output, weights):  # [B, D, P], [D, P]
        B, D, P = qsvt_output.shape
        results = []
        for b in range(B):
            vals = []
            for i in range(D):
                acc = []
                for p in range(P):
                    acc.append(lcu_circuit(qsvt_output[b, i, p], weights[i, p]))
                vals.append(torch.stack(acc))  # [P]
            results.append(torch.stack(vals))  # [D, P]
        return torch.stack(results)  # [B, D, P]

    return forward_fn

def make_quantum_sum_layer():
    dev = qml.device("default.qubit", wires=3)

    @qml.qnode(dev, interface="torch", diff_method="backprop")
    def quantum_sum(val1, val2):
        qml.Hadamard(wires=0)
        qml.RY(val1, wires=1)
        qml.RY(val2, wires=2)
        qml.CNOT(wires=[1, 0])
        qml.CNOT(wires=[2, 0])
        return qml.expval(qml.PauliZ(0))

    def forward_fn(lcu_output):  # [B, D, P] where P == 2
        B, D, P = lcu_output.shape
        assert P == 2, "Quantum sum layer assumes exactly P=2 polynomials"
        summed = []
        for b in range(B):
            summed_d = []
            for d in range(D):
                val1 = lcu_output[b, d, 0]
                val2 = lcu_output[b, d, 1]
                summed_d.append(quantum_sum(val1, val2))
            summed.append(torch.stack(summed_d))
        return torch.stack(summed)  # [B, D]

    return forward_fn

class SineKANLayer(nn.Module):
    def __init__(self, input_dim, output_dim, grid_size=5):
        super().__init__()
        self.grid_size = grid_size
        self.A, self.K, self.C = 0.9724, 0.9884, 0.9994
        self.grid_norm = (torch.arange(grid_size, dtype=torch.float32) + 1).reshape(1, 1, grid_size)
        amp = torch.empty(output_dim, input_dim, 1).normal_(0, .4) / output_dim / self.grid_norm
        self.amplitudes = nn.Parameter(amp.float())
        self.freq = nn.Parameter(torch.arange(1, grid_size + 1).float().reshape(1, 1, 1, grid_size))
        grid_phase = torch.arange(1, grid_size + 1, dtype=torch.float32).reshape(1, 1, 1, grid_size) / (grid_size + 1)
        input_phase = torch.linspace(0, math.pi, input_dim, dtype=torch.float32).reshape(1, 1, input_dim, 1)
        phase = grid_phase + input_phase
        for _ in range(1, grid_size):
            phase = (self.A * grid_size ** (-self.K) + self.C) * phase
        self.register_buffer("phase", phase)
        self.bias = nn.Parameter(torch.ones(1, output_dim).float() / output_dim)

    def forward(self, x):
        x = x.to(self.amplitudes.dtype)
        x = x.view(x.shape[0], -1).unsqueeze(-1).unsqueeze(1)
        s = torch.sin(x * self.freq + self.phase)
        y = torch.einsum('bijd,ojd->bo', s, self.amplitudes)
        return y + self.bias

class SineKAN(nn.Module):
    def __init__(self, input_dim, hidden=64, output_dim=2):
        super().__init__()
        self.l1 = SineKANLayer(input_dim, hidden)
        self.l2 = SineKANLayer(hidden, output_dim)

    def forward(self, x):
        return self.l2(self.l1(x))

class QSVT_LCU_QuantumSum_SineKAN_Model(nn.Module):
    def __init__(self, input_dim, coeff_list):
        super().__init__()
        self.qsvt = QSVTLayerMulti(coeff_list)
        self.lcu_weights = nn.Parameter(torch.randn(input_dim, len(coeff_list)))
        self.quantum_lcu = make_quantum_lcu_layer()
        self.quantum_sum = make_quantum_sum_layer()
        self.kan = SineKAN(input_dim=input_dim, hidden=64, output_dim=2)

    def forward(self, x):
        qsvt_out = self.qsvt(x)                            # [B, D, P]
        lcu_out = self.quantum_lcu(qsvt_out, self.lcu_weights)  # [B, D, P]
        quantum_summed = self.quantum_sum(lcu_out)              # [B, D]
        return self.kan(quantum_summed)


def load_higgs_data(path, Nsamples=1000):
    df = pd.read_csv(path, header=None)
    df = df.sample(frac=1.0, random_state=42)
    X = df.iloc[:Nsamples, 1:].values
    y = df.iloc[:Nsamples, 0].values
    X_min = X.min(axis=1, keepdims=True)
    X_max = X.max(axis=1, keepdims=True)
    X_scaled = 2 * (X - X_min) / (X_max - X_min + 1e-8) - 1
    return train_test_split(
        torch.tensor(X_scaled, dtype=torch.float32),
        torch.tensor(y, dtype=torch.long),
        test_size=0.2,
        stratify=y,
        random_state=42
    )

def train_model(file_path):
    X_train, X_test, y_train, y_test = load_higgs_data(file_path, Nsamples=1000)
    input_dim = X_train.shape[1]

    coeff_list = [
        [0.0, 1.0, 0.0, -0.5],      
        [0.0, 0.8, 0.0, 0.3]         
    ]

    model = QSVT_LCU_QuantumSum_SineKAN_Model(input_dim, coeff_list)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    X_train, X_test = X_train.to(device), X_test.to(device)
    y_train, y_test = y_train.to(device), y_test.to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(30):
        model.train()
        optimizer.zero_grad()
        preds = model(X_train)
        loss = criterion(preds, y_train)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            acc = (model(X_test).argmax(dim=1) == y_test).float().mean().item()
        print(f"[QSVT+LCU+Sum+SineKAN] Epoch {epoch+1:02d} | Loss: {loss.item():.4f} | Test Acc: {acc:.4f}")

train_sinekan_higgs(file_path)

[SineKAN HIGGS] Epoch 01 | Loss: 1.4266 | Test Acc: 0.5250
[SineKAN HIGGS] Epoch 02 | Loss: 0.7333 | Test Acc: 0.5000
[SineKAN HIGGS] Epoch 03 | Loss: 0.7044 | Test Acc: 0.5300
[SineKAN HIGGS] Epoch 04 | Loss: 0.6920 | Test Acc: 0.5350
[SineKAN HIGGS] Epoch 05 | Loss: 0.6864 | Test Acc: 0.5550
[SineKAN HIGGS] Epoch 06 | Loss: 0.6832 | Test Acc: 0.5500
[SineKAN HIGGS] Epoch 07 | Loss: 0.6795 | Test Acc: 0.5350
[SineKAN HIGGS] Epoch 08 | Loss: 0.6779 | Test Acc: 0.5450
[SineKAN HIGGS] Epoch 09 | Loss: 0.6759 | Test Acc: 0.5450
[SineKAN HIGGS] Epoch 10 | Loss: 0.6740 | Test Acc: 0.5650
[SineKAN HIGGS] Epoch 11 | Loss: 0.6708 | Test Acc: 0.5650
[SineKAN HIGGS] Epoch 12 | Loss: 0.6632 | Test Acc: 0.5750
[SineKAN HIGGS] Epoch 13 | Loss: 0.6551 | Test Acc: 0.5750
[SineKAN HIGGS] Epoch 14 | Loss: 0.6479 | Test Acc: 0.5900
[SineKAN HIGGS] Epoch 15 | Loss: 0.6393 | Test Acc: 0.5950
[SineKAN HIGGS] Epoch 16 | Loss: 0.6340 | Test Acc: 0.6050
[SineKAN HIGGS] Epoch 17 | Loss: 0.6260 | Test Acc: 0.59

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, input_dim, output_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.LayerNorm(128),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.LayerNorm(64),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        return self.net(x)


def load_higgs_data(path, Nsamples=1000):
    df = pd.read_csv(path, header=None)
    df = df.sample(frac=1.0, random_state=42)
    X = df.iloc[:Nsamples, 1:].values
    y = df.iloc[:Nsamples, 0].values
    X_min = X.min(axis=1, keepdims=True)
    X_max = X.max(axis=1, keepdims=True)
    X_scaled = 2 * (X - X_min) / (X_max - X_min + 1e-8) - 1
    return train_test_split(
        torch.tensor(X_scaled, dtype=torch.float32),
        torch.tensor(y, dtype=torch.long),
        test_size=0.2,
        stratify=y,
        random_state=42
    )


def train_mlp_model(file_path):
    X_train, X_test, y_train, y_test = load_higgs_data(file_path, Nsamples=1000)
    input_dim = X_train.shape[1]

    model = MLPModel(input_dim=input_dim, output_dim=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    X_train, X_test = X_train.to(device), X_test.to(device)
    y_train, y_test = y_train.to(device), y_test.to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        preds = model(X_train)
        loss = criterion(preds, y_train)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            acc = (model(X_test).argmax(dim=1) == y_test).float().mean().item()
        print(f"[MLP] Epoch {epoch+1:02d} | Loss: {loss.item():.4f} | Test Acc: {acc:.4f}")

train_mlp_model(file_path)


[MLP] Epoch 01 | Loss: 0.6973 | Test Acc: 0.4700
[MLP] Epoch 02 | Loss: 0.7248 | Test Acc: 0.5200
[MLP] Epoch 03 | Loss: 0.6844 | Test Acc: 0.5250
[MLP] Epoch 04 | Loss: 0.6873 | Test Acc: 0.5250
[MLP] Epoch 05 | Loss: 0.6812 | Test Acc: 0.5400
[MLP] Epoch 06 | Loss: 0.6766 | Test Acc: 0.5300
[MLP] Epoch 07 | Loss: 0.6715 | Test Acc: 0.5100
[MLP] Epoch 08 | Loss: 0.6633 | Test Acc: 0.4950
[MLP] Epoch 09 | Loss: 0.6616 | Test Acc: 0.5100
[MLP] Epoch 10 | Loss: 0.6495 | Test Acc: 0.5050
[MLP] Epoch 11 | Loss: 0.6340 | Test Acc: 0.5200
[MLP] Epoch 12 | Loss: 0.6225 | Test Acc: 0.5350
[MLP] Epoch 13 | Loss: 0.6029 | Test Acc: 0.5700
[MLP] Epoch 14 | Loss: 0.5945 | Test Acc: 0.5400
[MLP] Epoch 15 | Loss: 0.5667 | Test Acc: 0.5750
[MLP] Epoch 16 | Loss: 0.5599 | Test Acc: 0.5950
[MLP] Epoch 17 | Loss: 0.5421 | Test Acc: 0.5650
[MLP] Epoch 18 | Loss: 0.5434 | Test Acc: 0.5600
[MLP] Epoch 19 | Loss: 0.5300 | Test Acc: 0.5600
[MLP] Epoch 20 | Loss: 0.4948 | Test Acc: 0.5200
[MLP] Epoch 21 | Los